# ***Image Classification***

---




>  *Cloudy, Desert, Green Area, Water*


*  **Models:** ConvNeXt, Swin Transformer, VGG19 (transfer learning), Custom CNN
* **Framework:** TensorFlow / Keras




In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import os

In [2]:
# Extract the dataset
# Assuming the .zip file is in the /content directory
!unzip -o /content/DataSet.zip -d /content/  # Extract to /content, it should create DataSet folder

Archive:  /content/DataSet.zip
  inflating: /content/DataSet/DataSet/Data_cloudy/train_10021.jpg  
  inflating: /content/DataSet/DataSet/Data_cloudy/train_10070.jpg  
  inflating: /content/DataSet/DataSet/Data_cloudy/train_10096.jpg  
  inflating: /content/DataSet/DataSet/Data_cloudy/train_101.jpg  
  inflating: /content/DataSet/DataSet/Data_cloudy/train_10128.jpg  
  inflating: /content/DataSet/DataSet/Data_cloudy/train_10177.jpg  
  inflating: /content/DataSet/DataSet/Data_cloudy/train_10231.jpg  
  inflating: /content/DataSet/DataSet/Data_cloudy/train_10451.jpg  
  inflating: /content/DataSet/DataSet/Data_cloudy/train_10538.jpg  
  inflating: /content/DataSet/DataSet/Data_cloudy/train_10552.jpg  
  inflating: /content/DataSet/DataSet/Data_cloudy/train_10598.jpg  
  inflating: /content/DataSet/DataSet/Data_cloudy/train_10609.jpg  
  inflating: /content/DataSet/DataSet/Data_cloudy/train_10621.jpg  
  inflating: /content/DataSet/DataSet/Data_cloudy/train_10661.jpg  
  inflating: /conte

In [3]:
# Check the contents of the extracted directory
!ls /content/DataSet.zip

/content/DataSet.zip


In [4]:
#  Data Loading and Preparation
# ----------------------------------------------

# Use image_dataset_from_directory for simplicity
dataset_dir = '/content/DataSet/DataSet'  # Corrected path to the directory containing class subdirectories

# Split dataset into train/val/test
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.3,
    subset="training",
    seed=123,
    image_size=(224, 224),  # Standard input size for ConvNeXt, Swin, VGG19
    batch_size=32
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.15,
    subset="validation",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_dir,
    validation_split=0.15,
    subset="validation",
    seed=456,  # Different seed to simulate test split
    image_size=(224, 224),
    batch_size=32
)
class_names = train_ds.class_names

Found 1800 files belonging to 4 classes.
Using 1260 files for training.
Found 1800 files belonging to 4 classes.
Using 270 files for validation.
Found 1800 files belonging to 4 classes.
Using 270 files for validation.


In [5]:
# Prefetch for performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)


In [6]:
# Helper: Plot Metrics + Confusion Matrix
# ----------------------------------------------
def plot_metrics(history, title):
    plt.figure(figsize=(12,5))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.title(f'{title} Accuracy')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title(f'{title} Loss')
    plt.legend()
    plt.show()

def compute_specificity(cm):
    """
    Compute specificity for each class from confusion matrix.
    Specificity = TN / (TN + FP)
    """
    specificity = []
    for i in range(len(cm)):
        tn = np.sum(np.delete(np.delete(cm, i, axis=0), i, axis=1))
        fp = np.sum(np.delete(cm, i, axis=0)[:, i])
        spec = tn / (tn + fp) if (tn + fp) > 0 else 0
        specificity.append(spec)
    return specificity

def plot_confusion(y_true, y_pred, labels):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()


In [7]:
# ConvNeXt Model
# ----------------------------------------------
def build_convnext():
    base = keras.applications.ConvNeXtTiny(weights='imagenet', include_top=False, input_shape=(224,224,3))
    base.trainable = False
    model = keras.Sequential([
        data_augmentation,
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(len(class_names), activation='softmax')
    ])
    return model


In [8]:
# Swin Transformer Model
# ----------------------------------------------
def build_swin():

    base = keras.applications.SwinTransformerTiny(weights='imagenet', include_top=False, input_shape=(224,224,3))
    base.trainable = False
    model = keras.Sequential([
        data_augmentation,
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(len(class_names), activation='softmax')
    ])
    return model

In [9]:
# VGG19 Transfer Learning
# ----------------------------------------------
def build_vgg19():
    base = keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(224,224,3))
    base.trainable = True  # fine-tune
    model = keras.Sequential([
        data_augmentation,
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(len(class_names), activation='softmax')
    ])
    return model

In [10]:
# Custom CNN
# ----------------------------------------------
def build_custom():
    inputs = keras.Input(shape=(224,224,3))
    x = data_augmentation(inputs)
    for filters in [32, 64, 128]:
        x = layers.Conv2D(filters, (3,3), padding='same')(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)
        x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(len(class_names), activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    return model

In [11]:
# Training + Evaluation
# ----------------------------------------------
def train_and_evaluate(model, title):
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(train_ds, validation_data=val_ds, epochs=10)
    plot_metrics(history, title)

    # Evaluate on test
    y_true = []
    y_pred = []
    for imgs, labels in test_ds:
        preds = model.predict(imgs)
        y_true.extend(labels.numpy())
        y_pred.extend(np.argmax(preds, axis=1))

    plot_confusion(y_true, y_pred, class_names)
    print(classification_report(y_true, y_pred, target_names=class_names))

    # Compute and print specificity
    cm = confusion_matrix(y_true, y_pred) # Calculate confusion matrix
    spec = compute_specificity(cm)
    print("\nSpecificity (per class):")
    for lbl, s in zip(class_names, spec):
        print(f"{lbl}: {s:.2f}")

In [12]:
# Data Augmentation
# ----------------------------------------------
data_augmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
])

In [ ]:
# Build, train, and evaluate the Custom CNN model
custom_model = build_custom()
train_and_evaluate(custom_model, "Custom CNN")

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 15s 159ms/step - accuracy: 0.7689 - loss: 4.7962 - val_accuracy: 0.6778 - val_loss: 38.4918
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 15s 144ms/step - accuracy: 0.8483 - loss: 1.0074 - val_accuracy: 0.7148 - val_loss: 5.3254
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 135ms/step - accuracy: 0.8961 - loss: 0.2715 - val_accuracy: 0.7000 - val_loss: 3.8069
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 136ms/step - accuracy: 0.8638 - loss: 0.2659 - val_accuracy: 0.7185 - val_loss: 1.5080
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 137ms/step - accuracy: 0.8662 - loss: 0.6466 - val_accuracy: 0.8000 - val_loss: 0.3621
Epoch 6/10
26/40 ━━━━━━━━━━━━━━━━━━━━ 1s 130ms/step - accuracy: 0.9170 - loss: 0.4737

In [ ]:
def generate_adversarial(model, image, label, epsilon=0.01):
    image = tf.convert_to_tensor(image)
    label = tf.convert_to_tensor([label])
    with tf.GradientTape() as tape:
        tape.watch(image)
        prediction = model(image)
        loss = tf.keras.losses.sparse_categorical_crossentropy(label, prediction)
    gradient = tape.gradient(loss, image)
    perturbation = epsilon * tf.sign(gradient)
    adversarial_image = image + perturbation
    return tf.clip_by_value(adversarial_image, 0, 1)

# model = build_convnext()
# train_and_evaluate(model, "ConvNeXt")
# model = build_swin()
# train_and_evaluate(model, "Swin Transformer")
# model = build_vgg19()
# train_and_evaluate(model, "VGG19")
# model = build_custom()
# train_and_evaluate(model, "Custom CNN")


In [ ]:
model = build_convnext()

In [ ]:
train_and_evaluate(model, "ConvNeXt")

In [ ]:
def build_swin():
    # Replace SwinTransformerTiny with a commonly available model like ResNet50
    base = keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
    base.trainable = False
    model = keras.Sequential([
        data_augmentation,
        base,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4, activation='softmax') # Assuming 4 classes
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

model = build_swin()

In [ ]:
train_and_evaluate(model, "Swin Transformer")

In [ ]:
model = build_vgg19()

In [ ]:
train_and_evaluate(model, "VGG19")

In [ ]:
model = build_custom()

In [ ]:
train_and_evaluate(model, "Custom CNN")

In [ ]:
# Optional to compare the performance of all these models and conclude results
# ---------------------------------------------------------------------------------

from sklearn.metrics import classification_report
import pandas as pd

# Dictionary to store results
results = {}

def train_and_evaluate_collect(model_fn, title):
    print(f"\n Training {title}...")
    model = model_fn()
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Fit model
    history = model.fit(train_ds, validation_data=val_ds, epochs=10, verbose=0)

    # Evaluate
    y_true, y_pred = [], []
    for imgs, labels in test_ds:
        preds = model.predict(imgs, verbose=0)
        y_true.extend(labels.numpy())
        y_pred.extend(np.argmax(preds, axis=1))

    report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

    # Collect macro avg
    macro_avg = report['macro avg']

    results[title] = {
        'accuracy': report['accuracy'],
        'precision': macro_avg['precision'],
        'recall': macro_avg['recall'],
        'f1-score': macro_avg['f1-score']
    }

    # Optional: print detailed report
    print(classification_report(y_true, y_pred, target_names=class_names))

#  Run all models
train_and_evaluate_collect(build_convnext, "ConvNeXt")
train_and_evaluate_collect(build_swin, "Swin Transformer")
train_and_evaluate_collect(build_vgg19, "VGG19")
train_and_evaluate_collect(build_custom, "Custom CNN")

#  Display comparison
df_results = pd.DataFrame(results).T
print("\n Model Comparison:\n")
print(df_results)

# If you want, plot as bar chart
df_results.plot(kind='bar', figsize=(10,6), title='Model Performance Comparison (Macro Avg)')
